# Table of content
* [Imports and loading dataset](#import-load)
* [Exploration of the dataset](#data-exploration)
* [Engineering the features](#feature-engineering)
* [Model creation and fitting](#model-learning)

    -[imports and functions](#model-learning-top)
    
    -[setup](#model-learning-setup)

# IMPORT & LOAD<a class="anchor" id="import-load"></a>

In [24]:
####Import libraries
# SQL
import sqlite3
# Data Manipulation
import numpy as np
import pandas as pd
# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
#Nice Tables
from ipy_table import *

# Import and suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [25]:
#load data (make sure you have downloaded database.sqlite)
with sqlite3.connect(r'C:/Users/ernest.chocholowski/Desktop/Datasets/Soccer/database.sqlite') as con:
    country_df = pd.read_sql_query("SELECT * from Country", con)
    matches_df = pd.read_sql_query("SELECT * from Match", con)
    league_df = pd.read_sql_query("SELECT * from League", con)
    team_df = pd.read_sql_query("SELECT * from Team", con)
    player_df = pd.read_sql_query("select * from Player", con)

In [26]:
all_df = [country_df, matches_df, league_df, team_df, player_df]
country_df.dfname = 'country_df'
matches_df.dfname = 'matches_df'
league_df.dfname = 'league_df' 
team_df.dfname = 'team_df' 
player_df.dfname = 'player_df'

# DATA EXPLORATION<a class="anchor" id="data-exploration"></a>

In [27]:
for df in all_df:
    print(df.dfname)
    print(df.info())
    print('-'*40)

country_df
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
id      11 non-null int64
name    11 non-null object
dtypes: int64(1), object(1)
memory usage: 256.0+ bytes
None
----------------------------------------
matches_df
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Columns: 115 entries, id to BSA
dtypes: float64(96), int64(9), object(10)
memory usage: 22.8+ MB
None
----------------------------------------
league_df
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
id            11 non-null int64
country_id    11 non-null int64
name          11 non-null object
dtypes: int64(2), object(1)
memory usage: 344.0+ bytes
None
----------------------------------------
team_df
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 5 columns):
id                  299 non-null int64
team_api_id         299 non-null int64
tea

# FEATURE ENGINEERING<a class="anchor" id="feature-engineering"></a>

In [8]:
#preprocessing libraries
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, Imputer, OneHotEncoder
from imblearn.over_sampling import SMOTE

In [ ]:
def impute_numeric (dataset, formula):
    """
    Impute numeric values in a dataset usinng linear regression
    dataset = Pandas Dataframe
    formula e.g. 'Y ~ X1 + X2'
    """
    import statsmodels.formula.api as smf
    import pandas as pd
    
    lm = smf.ols(formula = formula, data = dataset)
    res = lm.fit()
    
    temp_train = dataset[pd.isnull(dataset).any(axis=1)].copy()
    temp_train = temp_train.drop(formula.split(None, 1)[0], axis=1).copy()
    
    var_pred = res.predict(temp_train)
    var_pred = var_pred.round(decimals=0)
    
    dataset[formula.split(None, 1)[0]].fillna(var_pred, inplace=True)

In [31]:
def label_win (row):
    if row['home_team_goal'] > row['away_team_goal']:
        return 'WIN'
    if row['home_team_goal'] == row['away_team_goal']:
        return 'DRAW'
    if row['home_team_goal'] < row['away_team_goal']:
        return 'LOSE'

matches_df['RESULT'] = matches_df.apply(lambda row: label_win(row), axis=1)

In [55]:
##################VERY BAD########################
matches_df.dropna(inplace=True)

#labelling
var_mod = ['id', 'country_id', 'league_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal', 'away_team_goal', 'home_player_X1',
       'home_player_X2', 'home_player_X3', 'home_player_X4',
       'home_player_X5', 'home_player_X6', 'home_player_X7',
       'home_player_X8', 'home_player_X9', 'home_player_X10',
       'home_player_X11', 'away_player_X1', 'away_player_X2',
       'away_player_X3', 'away_player_X4', 'away_player_X5',
       'away_player_X6', 'away_player_X7', 'away_player_X8',
       'away_player_X9', 'away_player_X10', 'away_player_X11',
       'home_player_Y1', 'home_player_Y2', 'home_player_Y3',
       'home_player_Y4', 'home_player_Y5', 'home_player_Y6',
       'home_player_Y7', 'home_player_Y8', 'home_player_Y9',
       'home_player_Y10', 'home_player_Y11', 'away_player_Y1',
       'away_player_Y2', 'away_player_Y3', 'away_player_Y4',
       'away_player_Y5', 'away_player_Y6', 'away_player_Y7',
       'away_player_Y8', 'away_player_Y9', 'away_player_Y10',
       'away_player_Y11', 'home_player_1', 'home_player_2',
       'home_player_3', 'home_player_4', 'home_player_5', 'home_player_6',
       'home_player_7', 'home_player_8', 'home_player_9', 'home_player_10',
       'home_player_11', 'away_player_1', 'away_player_2', 'away_player_3',
       'away_player_4', 'away_player_5', 'away_player_6', 'away_player_7',
       'away_player_8', 'away_player_9', 'away_player_10',
       'away_player_11', 'goal', 'shoton', 'shotoff', 'foulcommit', 'card',
       'cross', 'corner', 'possession', 'B365H', 'B365D', 'B365A', 'BWH',
       'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH',
       'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH',
       'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA', 'RESULT']
le = LabelEncoder()
for i in var_mod:
    mask = ~train_df[i].isnull()
    train_df[i][mask] = le.fit_transform(train_df[i][mask])

TypeError: 'NoneType' object is not subscriptable

# MODEL LEARNING<a class="anchor" id="model-learning"></a>

## IMPORTS AND FUNCTIONS<a class="anchor" id="model-learning-top"></a>

In [48]:
# machine learning models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [49]:
#Split data for train and test
def split_data(data, targ):
    #set target for training
    target = data[targ]

    # Import the train_test_split method
    from sklearn.model_selection import train_test_split
    # Split data into train (2/3rd of data) and test (1/3rd of data)
    return train_test_split(data, target, train_size = 0.75, random_state=0);

In [50]:
def regr_equation(logreg, train, target):
    if type(model) is LogisticRegression:
        coef = logreg.coef_[0]
        intercept = "{:.2f}".format(logreg.intercept_[0])
    else:
        coef = logreg.coef_
        intercept = "{:.2f}".format(logreg.intercept_)
        
    output = target.name + ' = ' + str(intercept) + ' + '
    coeff_df = pd.DataFrame(train.columns.delete(0))
    coeff_df.columns = ['Feature']
    coeff_df["Correlation"] = pd.Series(logreg.coef_[0])
    features = coeff_df['Feature'].tolist()
    coefficients = coeff_df['Correlation'].tolist()
    
    for coeff, feature in zip(coefficients, features):
        coeff_str = "{:.2f}".format(coeff)
        output += coeff_str + "*" + str(feature) + " + "
    return output[:-3]

In [51]:
def confusion_matrix(model, X, y):
    from sklearn.metrics import confusion_matrix
    return confusion_matrix(y, model.predict(X))

In [52]:
def test_model (model, data, target, submission_name = None, test=None):
   
    if test is None:
        from sklearn.metrics import accuracy_score, confusion_matrix, r2_score
        train, test, target_train, target_test = split_data(data, target)
        model.fit(train, target_train)
        #Calc parameters
        if type(model) is LogisticRegression:
            function_str = regr_equation(model, train, target_train)
        elif type(model) is LinearRegression:
            function_str = regr_equation(model, train, target_train)
        else :
            function_str = "NA"    

        if type(model) is not LinearRegression:
            trainset_acc = round(accuracy_score(target_train, model.predict(train)) * 100, 2)
            testset_acc = round(accuracy_score(target_test, model.predict(test)) * 100, 2)
            conf_matrix = confusion_matrix(target_train, model.predict(train))
            conf_matrix = 'TN: '+str(conf_matrix[0][0])+', FP: '+str(conf_matrix[0][1])+ \
                              ', FN: '+str(conf_matrix[1][0])+', TP: '+str(conf_matrix[1][1])
        else:
            trainset_acc = 'NA'
            testset_acc = 'NA'
            conf_matrix = "NA"

        r2_score = r2_score(target_train, model.predict(train))
        kaggle = "not_tested"

        #prints
        print("-"*40)
        print('Submission name:', submission_name )
        print('Regression function:\n', function_str)
        print('Accuracy on train set:', trainset_acc)
        print('Accuracy on test set:', testset_acc)
        print("R2 score:", r2_score)
        print("Confusion matrix:\n", conf_matrix)
        return [submission_name, function_str, str(trainset_acc), str(testset_acc), r2_score, conf_matrix, kaggle]

## SETUP<a class="anchor" id="model-learning-setup"></a>

In [53]:
#Setup
suffix = '_firstTry'
from sklearn.neural_network import MLPClassifier
modelMLP = MLPClassifier()
modelLogReg = LogisticRegression()
modelLinReg = LinearRegression()
modelSVC = SVC()
modellinSVC = LinearSVC()
modelKN = KNeighborsClassifier(n_neighbors = 3)
modelGNB = GaussianNB()
modelPercp = Perceptron()
modelSGD = SGDClassifier()
modelTree = DecisionTreeClassifier()
modelRndForest = RandomForestClassifier()

modelXTree = ExtraTreesClassifier()

models = {
          modelRndForest: "RandomForest",
          modelLogReg: "LogisticRegr"
         }

In [54]:
outputs = []
for model, prefix in models.items():
    name=prefix+suffix
    outputs.append(test_model(model, matches_df, target='RESULT', submission_name=name))
outputs.sort(key=lambda x: x[0])

ValueError: Unknown label type: 'unknown'

In [ ]:
for out in outputs:
    super_table.append(out)

In [ ]:
make_table(super_table)
apply_theme('basic')

In [ ]:
#df = pd.DataFrame(super_table[1:], columns=super_table[0])
#df.to_csv(r'C:/Users/ernest.chocholowski/Desktop/GIT/Titanic/table_eCh.csv', 
#                      index=False)